In [5]:
from pykrx import stock
import pandas as pd
import numpy as np
import json

In [4]:
# tickers = stock.get_market_ticker_list(market="KOSDAQ")
# ticker_name = {stock.get_market_ticker_name(ticker):ticker for ticker in tickers}
# with open("KOSDAQ.json", "w") as of:
#     json.dump(ticker_name, of)

In [6]:
with open("KOSDAQ.json") as json_file:
    ticker_name = json.load(json_file)
stock.get_market_ohlcv("20231220", "20231220", ticker_name["하림"])

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2023-12-20,3860,4905,3860,4905,69835543,29.933775


In [7]:
df = stock.get_market_ohlcv("20220101", "20231220", ticker_name["하림"])

In [12]:
df["Range"] = df["고가"] - df["저가"]
df["RangePerVolume"] = (df["Range"] / df["거래량"]) * 100_000

In [16]:
df["RangePerVolume"].quantile(np.linspace(0.1,1.0,num=10))

0.1     4.851777
0.2     6.883213
0.3     8.491247
0.4    10.246392
0.5    11.832173
0.6    13.167124
0.7    15.264198
0.8    18.149730
0.9    21.199469
1.0    58.354406
Name: RangePerVolume, dtype: float64

In [17]:
df.tail(10)

,시가,고가,저가,종가,거래량,등락률,Range,RangePerVolume
날짜,,,,,,,,
2023-12-07,2910,2920,2900,2905,465264,0.172414,20,4.298635
2023-12-08,2880,2910,2875,2910,294613,0.172117,35,11.879992
2023-12-11,2940,2950,2910,2920,519164,0.343643,40,7.704694
2023-12-12,2930,2945,2895,2895,452493,-0.856164,50,11.049895
2023-12-13,2900,2930,2890,2895,305264,0.000000,40,13.103412
2023-12-14,2905,2910,2885,2900,290729,0.172712,25,8.599073
2023-12-15,2910,2920,2885,2905,337000,0.172414,35,10.385757
2023-12-18,2890,2910,2885,2905,408555,0.000000,25,6.119127
2023-12-19,3665,3775,3305,3775,91565444,29.948365,470,0.513294


In [34]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pio.renderers.default='browser'  

def get_candlestick_chart(df: pd.DataFrame):  

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=(f"Stock Price", "Volume")
    )

    fig.add_trace(
        go.Candlestick(
            x = df.index,
            open = df['시가'], 
            high = df['고가'],
            low = df['저가'],
            close = df['종가'],
            name = 'Candlestick chart'
        ),
        row = 1,
        col = 1
    ) 

    fig.add_trace(
        go.Bar(x = df.index, y = df["거래량"], name="Volume"),
        row=2,
        col=1
    )

    fig['layout']['xaxis2']['title'] = "Date"
    fig['layout']['yaxis']['title'] = "Price"
    fig['layout']['yaxis2']['title'] = "Volume"

    fig.update_xaxes(
        rangebreaks = [{'bounds': ['sat', 'mon']}],
        rangeslider_visible = False,
        ) 
    return fig

In [35]:
get_candlestick_chart(df)
